In [1]:
import sys
import csv
import unicodedata
import numpy as np
import pandas as pd

In [2]:
from pymystem3 import Mystem
from collections import Counter
from multiprocessing import Pool
from IPython.display import display
from ipywidgets import FloatProgress
from sklearn.pipeline import Pipeline
from parsers.text_utils import DefaultTextProcessor, Lemmatizer

In [4]:
csv.field_size_limit(sys.maxsize)

9223372036854775807

Разобьём процесс на две части — токенизацию документов (без фильтрации) и, собственно, лемматизацию.

In [87]:
%%time

# Проделаем токенизацию с сохранением промежуточного
# состояния в ruwiki.tonekized.csv.tmp

tokenizer = DefaultTextProcessor()

# 1361758 — предподсчитанное кол-во документов
f = FloatProgress(min=0, max=1361758)
display(f)

unused_char = '\U00037b84'
def strip_accents(s):
    s = s.replace("й", unused_char)
    return "".join((c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c) != "Mn")).replace(unused_char, "й")

with open("../datasets/ruwiki/ruwiki.plain.csv", "r") as infile:
    with open("ruwiki.tonekized.csv.tmp", "w") as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)
        count = 0
        cached_rows = []
        for title, text in reader:
            text = strip_accents(text)
            tokens = tokenizer.fit_transform(text)
            cached_rows.append((title, " ".join(tokens)))
            count += 1
            if count % 1000 == 0:
                writer.writerows(cached_rows)
                outfile.flush()
                f.value += len(cached_rows)
                cached_rows = []
        # Запишем оставшиеся строчки
        writer.writerows(cached_rows)
        f.value += len(cached_rows)

CPU times: user 38min 48s, sys: 15.2 s, total: 39min 4s
Wall time: 39min 4s


In [88]:
%%time

# Теперь сделаем лемматизацию всех документов при помощи pymystem3
# Распараллеливая процесс на N_PROCS процессоров

N_PROCS = 4

# 1361758 — предподсчитанное кол-во документов
f = FloatProgress(min=0, max=1361758)
display(f)

m = Mystem()

def lemmatize(text):
    return "".join(m.lemmatize(text)).strip()

with open("ruwiki.tonekized.csv.tmp", "r") as infile:
    with open("../datasets/ruwiki/ruwiki.lemmatized.csv", "w") as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)
        count = 0
        cached_titles = []
        cached_texts = []
        for title, text in reader:
            cached_titles.append(title)
            cached_texts.append(text)
            count += 1
            if count % 1000 == 0:
                with Pool(N_PROCS) as p:
                    lemmatized_texts = p.map(lemmatize, cached_texts)
                writer.writerows(zip(cached_titles, lemmatized_texts))
                outfile.flush()
                f.value += len(cached_titles)
                cached_texts = []
                cached_titles = []
        # Запишем оставшиеся строчки
        with Pool(N_PROCS) as p:
            lemmatized_texts = p.map(lemmatize, cached_texts)
        writer.writerows(zip(cached_titles, lemmatized_texts))
        f.value += len(cached_titles)

CPU times: user 5min 27s, sys: 1min 47s, total: 7min 14s
Wall time: 3h 33min 19s


Посчитаем размер словаря, из которого состоит неотфильтрованная лемматизированная выборка.

In [97]:
%%time

dictionary = Counter()

# 1361758 — предподсчитанное кол-во документов
f = FloatProgress(min=0, max=1361758)
display(f)

with open("../datasets/ruwiki/ruwiki.lemmatized.csv", "r") as infile:
    reader = csv.reader(infile)
    for title, text in reader:
        tokens = text.split()
        dictionary.update(tokens)
        f.value += 1

CPU times: user 36min, sys: 2min 35s, total: 38min 35s
Wall time: 37min 4s


In [153]:
dict_series = pd.DataFrame.from_dict(dictionary, orient="index")[0]

In [154]:
len(dict_series)

4577979

In [161]:
(dict_series > 10).value_counts()[True]

566937

Будем считать, что лемма должна встретиться более 10 раз в коллекции, чтобы мы положили её в словарь. Это сократит размер словаря в 8 раз от первоначального объёма.

Посмотрим на топ-50 слов:

In [174]:
top50_words = dict_series.sort_values(ascending=False)[:50].index
top50_words

Index(['в', 'и', 'год', 'на', 'с', 'быть', 'по', 'из', 'он', 'который', 'а',
       'к', 'не', 'что', 'от', 'для', 'за', '1', 'как', 'этот', 'свой',
       'также', '2', 'до', 'первый', 'время', 'о', 'его', 'после', 'они',
       'район', '3', 'один', 'то', 'становиться', 'при', 'г', 'город', '5',
       'примечание', 'ссылка', 'человек', 'м', '_', 'тот', 'область', 'во',
       'это', 'она', 'весь'],
      dtype='object')

Слов, которые могли бы иметь выраженную тематику, здесь почти нет, зато довольно много мусорных и общих слов. Будем выбрасывать слова, входящие в этот список, из документов.

Также будем фильтровать слова по стоп-словарю.

In [179]:
stop_words = set(map(str.strip, open("../datasets/ruwiki/stopwords.txt").readlines()))

In [220]:
%%time

common_words = dict_series[dict_series > 10].index

# 1361758 — предподсчитанное кол-во документов
f = FloatProgress(min=0, max=1361758)
display(f)

def accept_word(w):
    return w not in stop_words and w not in top50_words and w in common_words

with open("../datasets/ruwiki/ruwiki.lemmatized.csv", "r") as infile:
    with open("../datasets/ruwiki/ruwiki.filtered.csv", "w") as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)
        for title, text in reader:
            tokens = text.split()
            writer.writerow((title, " ".join(filter(accept_word, tokens))))
            f.value += 1

CPU times: user 47min 58s, sys: 2min 58s, total: 50min 57s
Wall time: 48min 54s


Наконец, превратим коллекцию с отфильтрованным словарём в файл UCI Bag-of-words.

In [ ]:
%%time

docs = []

# 1361758 — предподсчитанное кол-во документов
f = FloatProgress(min=0, max=1361758)
display(f)

with open("../datasets/ruwiki/ruwiki.filtered.csv", "r") as infile:
    reader = csv.reader(infile)
    for title, text in reader:
        doc = {"title": title, "modalities": {}}
        doc["modalities"]["text"] = text.split()
        docs.append(doc)
        f.value += 1

---